# Больше-меньше

(Задача из контеста по [лабе](http://cs.mipt.ru/algo/lessons/sem_1/02.base_python.html).)

Дано выражение вида "a x b", где a и b — натуральные числа, а x — знак сравнения: '<' или '>'.
Истинно ли данное выражение?

### Формат входных данных

В первой строке число a, во второй знак сравнения, в третьей - число b. Оба числа - натуральные.

### Формат выходных данных

Если выражение истинно, выведите YES, иначе NO. 

### Примеры

<br/>

<div style="display: inline-block">

| Вход | Выход   |
| ---  | ------- |
| 1    | YES     |
| <    |         |
| 2    |         |

| Вход | Выход   |
| ---  | ------- |
| 10   | NO      |
| >    |         |
| 100  |         |

</div>

# Решение

In [1]:
a = int(input())
sign = input()
b = int(input())

1
<
2


## 1. "Нормальный", или If-s 1

In [2]:
if sign == '<':
    if a < b:
        print('YES')
    else:
        print('NO')
else:
    if a > b:
        print('YES')
    else:
        print('NO')

YES


## 2. "Нормальный, или If-s 2

In [3]:
if a > b and sign == '>':
    print('YES')
elif a < b and sign == '<':
    print('YES')
else:
    print('NO')

YES


## 3. Eval

In [4]:
if eval(f'{a} {sign} {b}'):
    print('YES')
else:
    print('NO')

YES


*P.S.*

В данной задаче кажется более-менее нормальным, но вообще eval-ом пользоваться не рекомендуется, потому что это "мутно":

* https://realpython.com/python-eval-function/#minimizing-the-security-issues-of-eval
* https://stackoverflow.com/a/1832957/8094251
* https://stackoverflow.com/a/1834754/8094251

## 4. Через цикл

In [5]:
x = a
y = b

while x > 0 and y > 0:
    x -= 1
    y -= 1

if (x == 0 and sign == '<') or (y == 0 and sign == '>'):
    print('YES')
else:
    print('NO')

YES


## 5. Через функцию

In [6]:
def compare(x: int, y: int, sign: str) -> bool:
    return (a > b and sign == '>') or (a < b and sign == '<')


if compare(a, b, sign):
    print('YES')
else:
    print('NO')

YES


## 6. Через класс

In [7]:
class Comparator:
    @staticmethod
    def compare(x: int, y: int, sign: str) -> bool:
        return (a > b and sign == '>') or (a < b and sign == '<')


if Comparator.compare(a, b, sign):
    print('YES')
else:
    print('NO')

YES


## 7. Lexicographically, or Fully Stringy

In [8]:
x = bin(a)[2:]
y = bin(b)[2:]

max_len = max(len(x), len(y))
x, y = [
    '0' * (max_len - len(s)) + s
    for s in [x, y]
]

if (x < y and sign == '<') or (x > y and sign == '>'):
    print('YES')
else:
    print('NO')

YES


## 8*. AI

Пример основан на примерах отсюда:
* https://huggingface.co/openchat/openchat-3.6-8b-20240522
* https://huggingface.co/microsoft/Phi-3.5-mini-instruct

In [ ]:
# Install PyTorch: https://pytorch.org/get-started/locally

! pip install transformers
! pip install accelerate

In [10]:
import os

# Turn off CUDA (we don't need it, and probably you don't have it)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [12]:
model_id = "microsoft/Phi-3.5-mini-instruct"  # ~ 7-8 Gb

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful AI assistant."
    },
    {
        "role": "user",
        "content": (
            f"Дано логическое выражение: {a} {sign} {b}."
            f" Если выражение истинно, выведи YES, иначе NO."
        )
    },
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)
input_ids = input_ids.to(model.device)

outputs = model.generate(
    input_ids,
    do_sample=False,
    max_new_tokens=10
)
response = outputs[0][input_ids.shape[-1]:]

answer = tokenizer.decode(response, skip_special_tokens=True)

print(answer)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.


YES


*P.S.*

Можно посмотреть весь вход модели как текст:

```python
tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False,
)
```

Можно посмотреть весь выход:

```python
tokenizer.decode(outputs[0])
```

## 9*. "Obfuscated Lambda"

Не самый хороший пример на lambda-функцию (намеренно неоправданно сложный однострочник).

In [14]:
answer = 'YES' if (lambda x, y, s: x < y if s == '<' else x > y)(a, b, sign) else 'NO'

print(answer)

YES


*P.S.*

Более интересный не самый простой пример с lambda-функцией: [Set of all subsets](https://wiki.python.org/moin/Powerful%20Python%20One-Liners#Set_of_all_subsets).

## 10*. Linux

In [9]:
! if (( $a $sign $b )); then echo YES; else echo NO; fi

YES
